PART 1: MERGE ALL TABLES

Load All Cleaned Data

In [4]:
import pandas as pd
import numpy as np
import sqlite3 
import warnings
import os
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

print("MASTER DATASET + SQL DATABASE")

# load all cleaned datasets
print("]nLoading cleaned data")

orders = pd.read_csv('../data/processed/orders_clean.csv', parse_dates=[
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
])

customers = pd.read_csv('../data/processed/customers_clean.csv')
order_items = pd.read_csv('../data/processed/order_items_clean.csv')
order_payments = pd.read_csv('../data/processed/order_payments_clean.csv')

print(f"Orders: {orders.shape}")
print(f"Customers: {customers.shape}")
print(f"Order Items: {order_items.shape}")
print(f"Payments: {order_payments.shape}")

MASTER DATASET + SQL DATABASE
]nLoading cleaned data
Orders: (96478, 15)
Customers: (99441, 5)
Order Items: (112650, 20)
Payments: (99440, 4)


Merge Step 1: Orders + Customers

In [5]:
print("STEP 1: MERGING ORDERS + CUSTOMERS")

master_df = orders.merge(
    customers,
    on='customer_id',
    how='left',
)

print(f"\nOrders + Customers merged")
print(f"    Records: {len(master_df):,}")
print(f"    Columns: {master_df.shape[1]}")

# Validate merge
unmatched = master_df[master_df['customer_state'].isnull()]
if len(unmatched) == 0:
    print(f"\nValidation: All {len(master_df):,} orders have matching customers")
else:
    print(f"\nWarning: {len(unmatched):,} orders without matching customers")

STEP 1: MERGING ORDERS + CUSTOMERS

Orders + Customers merged
    Records: 96,478
    Columns: 19

Validation: All 96,478 orders have matching customers


Merge Step 2: Add Order Items

In [17]:
print("STEP 2: ADDING ORDER ITEMS")

# Check for overlapping columns
overlap = set(master_df.columns) & set(order_items.columns)
overlap.discard('order_id') # Remove join key from overlap
if len(overlap) > 0:
    print(f"Overlapping columns detected: {overlap}")
    print("    Using suffixes to handle overlap...")

# This will create multiple rows per order if order has multiple items
master_df = master_df.merge(
    order_items,
    on='order_id',
    how='left',
    suffixes=('', '_item')
)
print(f"\nOrder Items added")
print(f"    Records: {len(master_df):,}")
print(f"    Columns: {master_df.shape[1]}")
print(f"    Note: Row count increased because orders can have multiple items")

STEP 2: ADDING ORDER ITEMS
Overlapping columns detected: {'product_description_lenght', 'product_id', 'product_name_lenght', 'product_category_name_english', 'product_photos_qty', 'product_length_cm', 'has_dimensions', 'seller_id', 'freight_pct_of_price', 'product_category_name', 'product_height_cm', 'freight_value', 'product_width_cm', 'order_item_id', 'product_weight_g', 'price', 'total_item_cost', 'product_volume_cm3', 'shipping_limit_date'}
    Using suffixes to handle overlap...

Order Items added
    Records: 353,791
    Columns: 85
    Note: Row count increased because orders can have multiple items


Merge Step 3: Add Payments

In [19]:
print("STEP 3: ADDING PAYMENTS")

# Drop payment columns if they exist from previous run
payment_cols = ['total_payment_value', 'max_installments', 'payment_methods']
for col in payment_cols:
    if col in master_df.columns:
        master_df = master_df.drop(col, axis=1)
        print(f"Dropped existing column: {col}")
        
master_df = master_df.merge(
    order_payments,
    on='order_id',
    how='left'
)

print(f"\n✅ Payments added")
print(f"   Records: {len(master_df):,}")
print(f"   Columns: {master_df.shape[1]}")

# Check for orders without payment data
unmatched_payments = master_df[master_df['total_payment_value'].isnull()]
print(f"\n🔍 Validation:")
print(f"   Orders with payment data: {len(master_df) - len(unmatched_payments):,}")
print(f"   Orders without payment data: {len(unmatched_payments):,}")
if len(unmatched_payments) > 0:
    print(f"   (This is normal - cancelled orders may not have payments)")

STEP 3: ADDING PAYMENTS
Dropped existing column: total_payment_value
Dropped existing column: max_installments
Dropped existing column: payment_methods

✅ Payments added
   Records: 353,791
   Columns: 85

🔍 Validation:
   Orders with payment data: 353,764
   Orders without payment data: 27
   (This is normal - cancelled orders may not have payments)


Final Data Qality Check

In [21]:
print("MASTER DATASET QUALITY CHECK")

print(f"\nFinal Shape: {master_df.shape}")
print(f"    Total rowsL {len(master_df):,}")
print(f"    Total columns: {master_df.shape[1]}")

print(f"\nSample of columns (first 15):")
for i, col in enumerate(master_df.columns[:15], 1):
    print(f"    {i}. {col}")
print(f"    ... and {master_df.shape[1] - 15} more columns")

print(f"\nMissing Values (Top 10 Columns with Missing Data):")
missing = master_df.isnull().sum().sort_values(ascending=False).head(10)
has_missing = False
for col, count in missing.items():
    if count > 0:
        has_missing = True
        pct = (count / len(master_df)) * 100
        print(f"    {col}: {count:,} ({pct:.1f}%)") 
if not has_missing:
    print("    No missing values in top columns")

MASTER DATASET QUALITY CHECK

Final Shape: (353791, 85)
    Total rowsL 353,791
    Total columns: 85

Sample of columns (first 15):
    1. order_id
    2. customer_id
    3. order_status
    4. order_purchase_timestamp
    5. order_approved_at
    6. order_delivered_carrier_date
    7. order_delivered_customer_date
    8. order_estimated_delivery_date
    9. delivery_time_days
    10. delivery_delay_days
    11. on_time_delivery
    12. order_year
    13. order_month
    14. order_day_of_week
    15. order_hour
    ... and 70 more columns

Missing Values (Top 10 Columns with Missing Data):
    product_name_lenght_item: 4,096 (1.2%)
    product_photos_qty: 4,096 (1.2%)
    product_name_lenght: 4,096 (1.2%)
    product_description_lenght_item: 4,096 (1.2%)
    product_name_lenght_item: 4,096 (1.2%)
    product_description_lenght_item: 4,096 (1.2%)
    product_photos_qty_item: 4,096 (1.2%)
    product_description_lenght: 4,096 (1.2%)
    product_photos_qty_item: 4,096 (1.2%)
    product_

Business Metrics Summary

In [24]:
from email.utils import unquote


print("BUSINESS METRICS SUMMARY")

print("\nRevenue Metrics:")
total_revenue =  master_df['total_payment_value'].sum()
print(f"    Total revenue: R$ {total_revenue:,.2f}")

avg_order_value = master_df.groupby('order_id')['total_payment_value'].first().mean()
print(f"    Average order value: R$ {avg_order_value:,.2f}")

print("\nOrder Metrics:")
unique_orders = master_df['order_id'].nunique()
unique_customers = master_df['customer_unique_id'].nunique()
unique_products = master_df['product_id'].nunique()

print(f"    Unique Orders: {unique_orders:,}")
print(f"    Unique Customer: {unique_customers:,}")
print(f"    Unique Products: {unique_products:,}")
print(f"    Total Order Items: {len(master_df):,}")

print("\nGeographic Coverage:")
states = master_df['customer_state'].nunique()
cities = master_df['customer_city'].nunique()
print(f"    States: {states}")
print(f"    Cities: {cities:,}")

print("\nData Range:")
min_date = master_df['order_purchase_timestamp'].min()
max_date = master_df['order_purchase_timestamp'].max()
print(f"    From: {min_date}")
print(f"    ToL {max_date}")

days_span = (max_date - min_date).days
print(f"    Span: {days_span} days ({days_span/365:.1f} years)")

BUSINESS METRICS SUMMARY

Revenue Metrics:
    Total revenue: R$ 173,153,152.98
    Average order value: R$ 159.86

Order Metrics:
    Unique Orders: 96,478
    Unique Customer: 93,358
    Unique Products: 32,216
    Total Order Items: 353,791

Geographic Coverage:
    States: 27
    Cities: 4,085

Data Range:
    From: 2016-09-15 12:16:38
    ToL 2018-08-29 15:00:37
    Span: 713 days (2.0 years)


Save Master Dataset

In [27]:
print("SAVING MASTER DATASET")

master_df.to_csv('../data/processed/master_dataset.csv', index=False)

print("\nMaster dataset saved: ../data/processed/master_dataset.csv")
print(f"    Size: {len(master_df):,} rows x {master_df.shape[1]} columns")

# Get file size
file_size = os.path.getsize('../data/processed/master_dataset.csv')
size_mb = file_size / (1024 * 1024)
print(f"    File size: {size_mb:.1f} MB")

SAVING MASTER DATASET

Master dataset saved: ../data/processed/master_dataset.csv
    Size: 353,791 rows x 85 columns
    File size: 307.4 MB


PART 2: CREATE SQLITE DATABASE

Initialize Database Connection

In [28]:
print("CREATING SQLITE DATABASE")

# Connect to database (creates file if doesn't exist)
conn = sqlite3.connect('../data/processed/ecommerce.db')
cursor = conn.cursor()

print("Database connection established: ../data/processed/ecommerce.db")

CREATING SQLITE DATABASE
Database connection established: ../data/processed/ecommerce.db


 Create Orders Table Schema

In [30]:
print("\nCreating Table: orders")

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id TEXT PRIMARY KEY,
    customer_id TEXT NOT NULL,
    order_status TEXT,
    order_purchase_timestamp TIMESTAMP,
    order_approved_at TIMESTAMP,
    order_delivered_carrier_date TIMESTAMP,
    order_delivered_customer_date TIMESTAMP,
    order_estimated_delivery_date TIMESTAMP,
    delivery_time_days INTEGER,
    delivery_delay_days INTEGER,
    on_time_delivery INTEGER,
    order_year INTEGER,
    order_month INTEGER,
    order_day_of_week INTEGER,
    order_hour INTEGER
)
''')

conn.commit()
print("Orders table schema created")


Creating Table: orders
Orders table schema created


Create Customers Table Schema

In [32]:
print("\nCreating Table: customers")

cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id TEXT PRIMARY KEY,
    customer_unique_id TEXT,
    customer_zip_code_prefix INTEGER,
    customer_city TEXT,
    customer_state TEXT
)
''')

conn.commit()
print("Customers table schema created")


Creating Table: customers
Customers table schema created


Create Products Table Schema

In [37]:
print("\nCreating Table: products")

# Load products
products_clean = pd.read_csv('../data/processed/products_clean.csv')

cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    product_id TEXT PRIMARY KEY,
    product_category_name TEXT,
    product_category_name_english TEXT,
    product_weight_g REAL,
    product_length_cm REAL,
    product_height_cm REAL,
    product_width_cm REAL,
    product_volume_cm3 REAL,
    has_dimensions INTEGER
)
''')

conn.commit()
print("Products table schema created")


Creating Table: products


Products table schema created


Create Order Items Table Schema

In [40]:
print("\nCreating Table: order_items")

cursor.execute('''
CREATE TABLE IF NOT EXISTS order_items (
    order_id TEXT,
    order_item_id INTEGER,
    product_id TEXT,
    seller_id TEXT,
    shipping_limit_date TIMESTAMP,
    price REAL,
    freight_value REAL,
    total_item_cost REAL,
    freight_pct_of_price REAL,
    PRIMARY KEY (order_id, order_item_id),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')

conn.commit()
print("Order Items table schema created")


Creating Table: order_items
Order Items table schema created


Create Order Payments Table Schema

In [41]:
print("\nCreating Table: order_payments")

cursor.execute('''
CREATE TABLE IF NOT EXISTS order_payments (
    order_id TEXT PRIMARY KEY,
    total_payment_value REAL,
    max_installments INTEGER,
    payment_methods TEXT,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
)
''')

conn.commit()
print("\nAll table schemas created and commited")


Creating Table: order_payments

All table schemas created and commited


Insert Data: Orders

In [42]:
print("INSERTING DATA INTO TABLE")

print("\nInserting orders")
orders.to_sql('orders', conn, if_exists='replace', index=False)
print(f"{len(orders):,} orders inserted")

INSERTING DATA INTO TABLE

Inserting orders
96,478 orders inserted


Insert Data: Customers

In [43]:
print("\nInserting customers...")
customers.to_sql('customers', conn, if_exists='replace', index=False)
print(f"{len(customers):,} customers inserted")


Inserting customers...
99,441 customers inserted


Insert Data: Products

In [44]:
print("\nInserting order products...")
products_clean.to_sql('products', conn, if_exists='replace', index=False)
print(f"{len(products_clean):,} products inserted")


Inserting order products...
32,951 products inserted


Insert Data: Order Items

In [67]:
print("\nInserting order items...")

# Select only relevant columns for database
order_items_for_db = order_items[[
    'order_id', 'order_item_id', 'product_id', 'seller_id',
    'shipping_limit_date', 'price', 'freight_value',
    'total_item_cost', 'freight_pct_of_price'
]].copy()

order_items_for_db.to_sql('order_items', conn, if_exists='replace', index=False)
print(f"{len(order_items_for_db):,} order items inserted")


Inserting order items...


112,650 order items inserted


Insert Data: Paymets

In [47]:
print("\nInserting payments...")
order_payments.to_sql('order_payments', conn, if_exists='replace', index=False)
print(f"{len(order_payments):,} payments inserted")

print("ALL DATA INSERTED SUCCESSFULLY")


Inserting payments...
99,440 payments inserted
ALL DATA INSERTED SUCCESSFULLY


PART 3: TEST DATABASE

Verify Table Creation

In [54]:
print("\nDATABASE VERIFICATION")

# List all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table'"
tables = pd.read_sql_query(tables_query, conn)

print("\nTAbles in database:")
for i, table in enumerate(tables['name'], 1):
    print(f"    {i}, {table}")


DATABASE VERIFICATION

TAbles in database:
    1, orders
    2, customers
    3, products
    4, order_items
    5, order_payments


Test Query 1: Row Counts

In [57]:
print("TEST 1: ROW COUNTS")

for table in tables['name']:
    count_query= f"SELECT COUNT(*) as count FROM {table}"
    result = pd.read_sql_query(count_query, conn)
    print(f"    {table}: {result['count'][0]:,} rows")

TEST 1: ROW COUNTS
    orders: 96,478 rows
    customers: 99,441 rows
    products: 32,951 rows
    order_items: 112,650 rows
    order_payments: 99,440 rows


Test Query 2: Sample Data

In [58]:
print("TEST 2: SAMPLE DATA FROM ORDERS")

sample_query = """
SELECT
    order_id,
    customer_id,
    order_status,
    order_purchase_timestamp,
    delivery_time_days,
    on_time_delivery
FROM orders
LIMIT 5
"""

sample = pd.read_sql_query(sample_query, conn)
print("\n", sample.to_string(index=False))

TEST 2: SAMPLE DATA FROM ORDERS

                         order_id                      customer_id order_status order_purchase_timestamp  delivery_time_days  on_time_delivery
e481f51cbdc54678b7cc49136f2d6af7 9ef432eb6251297304e76186b10a928d    delivered      2017-10-02 10:56:33                 8.0                 1
53cdb2fc8bc7dce0b6741e2150273451 b0830fb4747a6c6d20dea0b8c802d7ef    delivered      2018-07-24 20:41:37                13.0                 1
47770eb9100c2d0c44946d9cf07ec65d 41ce2a54c0b03bf3443c3d931a367089    delivered      2018-08-08 08:38:49                 9.0                 1
949d5b44dbf5de918fe9c16f97b45f8a f88197465ea7920adcdbec7375364d82    delivered      2017-11-18 19:28:06                13.0                 1
ad21c59c0840e6cb83a9ceb5573f8159 8ab97904e6daea8866dbdbc4fb7aad2c    delivered      2018-02-13 21:18:39                 2.0                 1


Test Query 3: Join Test

In [60]:
print("TEST 3: JOIN TEST (Orders + Customers)")

join_query = """
SELECT
    o.order_id,
    o.order_status,
    c.customer_city,
    c.customer_state
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
LIMIT 5
"""

join_result = pd.read_sql_query(join_query, conn)
print("\n", join_result.to_string(index=False))
print(f"\nJoin successful: {len(join_result)} rows returned")

TEST 3: JOIN TEST (Orders + Customers)

                         order_id order_status           customer_city customer_state
e481f51cbdc54678b7cc49136f2d6af7    delivered               sao paulo             SP
53cdb2fc8bc7dce0b6741e2150273451    delivered               barreiras             BA
47770eb9100c2d0c44946d9cf07ec65d    delivered              vianopolis             GO
949d5b44dbf5de918fe9c16f97b45f8a    delivered sao goncalo do amarante             RN
ad21c59c0840e6cb83a9ceb5573f8159    delivered             santo andre             SP

Join successful: 5 rows returned


 Test Query 4: Aggregation Test

In [61]:
print("TEST 4: AGGREGATION TEST")

agg_query = """
SELECT
    order_year,
    order_month,
    COUNT(*) as total_orders,
    AVG(delivery_time_days) as avg_delivery_days
FROM orders
WHERE delivery_time_days IS NOT NULL
GROUP BY order_year, order_month
LIMIT 10
"""

agg_result = pd.read_sql_query(agg_query, conn)
print("\n", agg_result.to_string(index=False))
print(f"\nAggregation successful: {len(agg_result)} rows returned")

TEST 4: AGGREGATION TEST

  order_year  order_month  total_orders  avg_delivery_days
       2016            9             1          54.000000
       2016           10           265          19.135849
       2016           12             1           4.000000
       2017            1           750          12.092000
       2017            2          1653          12.606171
       2017            3          2546          12.395522
       2017            4          2303          14.352584
       2017            5          3545          10.760508
       2017            6          3135          11.506220
       2017            7          3872          11.131198

Aggregation successful: 10 rows returned


PART 4: WRITE FIRST 3 QUERIE

Query 1: Total Revenue Summary

In [63]:
print("Query 1: TOTAL REVENUE SUMMARY")

query1 = """
SELECT
    COUNT(DISTINCT o.order_id) as total_orders,
    COUNT(DISTINCT o.customer_id) as total_customers,
    SUM(p.total_payment_value) as total_revenue,
    AVG(p.total_payment_value) as avg_order_value,
    MIN(o.order_purchase_timestamp) as firsr_order,
    MAX(o.order_purchase_timestamp) as last_order
FROM orders o
JOIN order_payments p ON o.order_id = p.order_id
"""

result1 = pd.read_sql_query(query1, conn)
print("\n", result1.to_string(index=False))

# Store for reporting
total_revenue = result1['total_revenue'][0]
total_orders = result1['total_orders'][0]
avg_order_value = result1['avg_order_value'][0]

print(f"\n METRICS:")
print(f"    Total Revenue: R$ {total_revenue:,.2f}")
print(f"    Total Orders: {total_orders:,}")
print(f"    Average Order Value: R$ {avg_order_value:,.2f}")

Query 1: TOTAL REVENUE SUMMARY

  total_orders  total_customers  total_revenue  avg_order_value         firsr_order          last_order
        96477            96477    15422461.77       159.856357 2016-10-03 09:44:50 2018-08-29 15:00:37

 METRICS:
    Total Revenue: R$ 15,422,461.77
    Total Orders: 96,477
    Average Order Value: R$ 159.86


Query 2: Revenue by State (Top 10)

In [68]:
print("Query 2: TOP 10 STATES BY REVENUE")

query2 = """
SELECT
    c.customer_state,
    COUNT(DISTINCT o.order_id) as total_orders,
    SUM(p.total_payment_value) as total_revenue,
    AVG(p.total_payment_value) as avg_order_value,
    ROUND(SUM(p.total_payment_value) * 100.0 /
            (SELECT SUM(total_payment_value) FROM order_payments), 2) as revenue_pct
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_payments p ON o.order_id = p.order_id
GROUP BY c.customer_state
ORDER BY total_revenue DESC
LIMIT 10
"""

result2 = pd.read_sql_query(query2, conn)
print("\n", result2.to_string(index=False))

print(f"\nINSIGHT:")
top_state = result2.iloc[0]
print(f"    {top_state['customer_state']} generates {top_state['revenue_pct']}% of total revenue")
print(f"    with {top_state['total_orders']:,.0f} orders")

Query 2: TOP 10 STATES BY REVENUE



 customer_state  total_orders  total_revenue  avg_order_value  revenue_pct
            SP         40500     5770266.19       142.475708        36.04
            RJ         12350     2055690.45       166.452668        12.84
            MG         11354     1819277.61       160.232307        11.36
            RS          5345      861802.40       161.235248         5.38
            PR          4923      781919.55       158.829890         4.88
            SC          3546      595208.40       167.853469         3.72
            BA          3256      591270.60       181.594165         3.69
            DF          2080      346146.17       166.416428         2.16
            GO          1957      334294.22       170.819734         2.09
            ES          1995      317682.65       159.239424         1.98

INSIGHT:
    SP generates 36.04% of total revenue
    with 40,500 orders


Query 3: Delivery Performance

In [70]:
print("Query 3: DELIVERY PERFORMANCE METRICS")

query3 = """
SELECT
    COUNT(*) as total_delivered_orders,
    ROUND(AVG(delivery_time_days), 1) as avg_delivery_days,
    MIN(delivery_time_days) as fastest_delivery,
    MAX(delivery_time_days) as slowest_delivery,
    ROUND(AVG(CAST(on_time_delivery AS FLOAT)) * 100) as on_time_pct,
    SUM(CASE WHEN delivery_delay_days > 0 THEN 1 ELSE 0 END) as late_deliveries,
    ROUND(SUM(CASE WHEN delivery_delay_days >0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as late_pct
FROM orders
WHERE delivery_time_days IS NOT NULL
"""

result3 = pd.read_sql_query(query3, conn)
print("\n", result3.to_string(index=False))

print(f"\nINSIGHT:")
avg_delivery = result3['avg_delivery_days'][0]
on_time_pct = result3['on_time_pct'][0]
late_pct = result3['late_pct'][0]

print(f"   Average delivery: {avg_delivery:.1f} days")
print(f"   On-time rate: {on_time_pct}%")
print(f"   Late delivery rate: {late_pct}%")

Query 3: DELIVERY PERFORMANCE METRICS

  total_delivered_orders  avg_delivery_days  fastest_delivery  slowest_delivery  on_time_pct  late_deliveries  late_pct
                  96470               12.1               0.0             209.0         93.0             6534      6.77

INSIGHT:
   Average delivery: 12.1 days
   On-time rate: 93.0%
   Late delivery rate: 6.77%


 Save Queries to SQL File

In [71]:
print("SAVING QUERIES TO SQL FILE")

queries_sql = f"""-- E-COMMERCE ANALYSIS - SQL QUERIES
-- Database: ecommerce.db
-- Created: January 4, 2026
-- Author: Christian Wedha

-- ============================================================

-- QUERY 1: Total Revenue Summary
-- Purpose: Overall business metrics

SELECT
    COUNT(DISTINCT o.order_id) as total_orders,
    COUNT(DISTINCT o.customer_id) as total_customers,
    SUM(p.total_payment_value) as total_revenue,
    AVG(p.total_payment_value) as avg_order_value,
    MIN(o.order_purchase_timestamp) as firsr_order,
    MAX(o.order_purchase_timestamp) as last_order
FROM orders o
JOIN order_payments p ON o.order_id = p.order_id

-- ============================================================

-- QUERY 2: Top 10 States by Revenue
-- Purpose: Geographic revenue analysis

SELECT
    c.customer_state,
    COUNT(DISTINCT o.order_id) as total_orders,
    SUM(p.total_payment_value) as total_revenue,
    AVG(p.total_payment_value) as avg_order_value,
    ROUND(SUM(p.total_payment_value) * 100.0 /
            (SELECT SUM(total_payment_value) FROM order_payments), 2) as revenue_pct
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_payments p ON o.order_id = p.order_id
GROUP BY c.customer_state
ORDER BY total_revenue DESC
LIMIT 10

-- ============================================================
-- QUERY 3: Delivery Performance Metrics
-- Purpose: Operational efficiency analysis

SELECT
    COUNT(*) as total_delivered_orders,
    ROUND(AVG(delivery_time_days), 1) as avg_delivery_days,
    MIN(delivery_time_days) as fastest_delivery,
    MAX(delivery_time_days) as slowest_delivery,
    ROUND(AVG(CAST(on_time_delivery AS FLOAT)) * 100) as on_time_pct,
    SUM(CASE WHEN delivery_delay_days > 0 THEN 1 ELSE 0 END) as late_deliveries,
    ROUND(SUM(CASE WHEN delivery_delay_days >0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as late_pct
FROM orders
WHERE delivery_time_days IS NOT NULL

-- ============================================================

"""
os.makedirs('sql', exist_ok=True)

with open('../sql/initial_queries.sql', 'w') as f:
    f.write(queries_sql)

print("Queries saved to: ../sql/initial_queries.sql")


SAVING QUERIES TO SQL FILE
Queries saved to: ../sql/initial_queries.sql


SUMMARY

In [ ]:
print("COMPLETE SUMMARY")

print("\nWhat we accomplished:")
print("    1. Merged 4 tables into master dataset")
print(f"    2. Created SQLite database with 5 tables")
print(f"    3. Inserted {len(orders):,} orders")
print(f"    4. Inserted {len(customers):,} customers")
print(f"    5. Inserted {len(products_clean):,} products")
print(f"    6. Inserted {len(order_items_for_db):,} order items")
print(f"    7. Inserted {len(order_payments):,} payments")
print("    8. Verified database with test queries")
print("    9. Wrote and saved first 3 business queries")

print("\nFiles created:")
print("    • data/processed/master_dataset.csv")
print("    • data/processed/ecommerce.db")
print("    • sql/initial_queries.sql")

print("\nKey Metrics (from queries):")
print(f"    • Total Revenue: R$ {total_revenue:,.2f}")
print(f"    • Total Orders: {total_orders:,}")
print(f"    • Average Order Value: R$ {avg_order_value:,.2f}")
print(f"    • Average Delivery: {avg_delivery:.1f} days")
print(f"    • On-Time Rate: {on_time_pct}%")

print("\nNext Steps (Day 5):")
print("    • Write 7 more business queries (total 10)")
print("    • Revenue analysis by category")
print("    • Customer segmentation queries")
print("    • Time-based analysis (day/hour patterns)")
print("    • Freight cost analysis")

# Close database connection
conn.close()
print("\nDatabase connection closed")